In [72]:
import cv2
import numpy as np

sigma = 0.33

img = cv2.imread('retina2/172.bmp')
# gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

smooth = cv2.imread('sample_172_201_Nwindow30.bmp')
# smooth = cv2.medianBlur(img, 5)

sobelX = cv2.Sobel(smooth, cv2.CV_64F, 1, 0)
sobelY = cv2.Sobel(smooth, cv2.CV_64F, 1, 0)
sobelX = np.uint8(np.absolute(sobelX))
sobelY = np.uint8(np.absolute(sobelY))
sobelCombined = cv2.bitwise_or(sobelX, sobelY)

# auto canny
v = np.median(smooth)
# apply automatic Canny edge detection using the computed median
lower = int(max(0, (1.0 - sigma) * v))
upper = int(min(255, (1.0 + sigma) * v))

# canny = cv2.Canny(smooth, lower, upper, 7)
print(lower, upper)
canny = cv2.Canny(smooth, 60, 79, 3)
canny = np.repeat(np.expand_dims(canny, axis=2), 3, axis=2)
canny = canny * (0.5, 0.5, 1.0)

cv2.imwrite('viz_outputs/blur.png', smooth)
cv2.imwrite('viz_outputs/canny.png', canny)
cv2.imwrite('viz_outputs/sobel.png', sobelCombined)

40 79


True